Q-classify the Size_Categorie using SVM

month	month of the year: 'jan' to 'dec'
day	day of the week: 'mon' to 'sun'
FFMC	FFMC index from the FWI system: 18.7 to 96.20
DMC	DMC index from the FWI system: 1.1 to 291.3
DC	DC index from the FWI system: 7.9 to 860.6
ISI	ISI index from the FWI system: 0.0 to 56.10
temp	temperature in Celsius degrees: 2.2 to 33.30
RH	relative humidity in %: 15.0 to 100
wind	wind speed in km/h: 0.40 to 9.40
rain	outside rain in mm/m2 : 0.0 to 6.4
Size_Categorie 	the burned area of the forest ( Small , Large)

In [140]:
#import libraby
import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split ,cross_val_score
import numpy as np

In [141]:
# load dataset
forest=pd.read_csv("E:\\DATA SCIENCE\\LMS\\ASSIGNMENT\\MY ASSIGNMENT\\SVM\\forestfires.csv")

In [142]:
# check head of dataset
forest.head(10)

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
5,aug,sun,92.3,85.3,488.0,14.7,22.2,29,5.4,0.0,...,0,0,0,0,0,0,0,0,0,small
6,aug,mon,92.3,88.9,495.6,8.5,24.1,27,3.1,0.0,...,0,0,0,0,0,0,0,0,0,small
7,aug,mon,91.5,145.4,608.2,10.7,8.0,86,2.2,0.0,...,0,0,0,0,0,0,0,0,0,small
8,sep,tue,91.0,129.5,692.6,7.0,13.1,63,5.4,0.0,...,0,0,0,0,0,0,0,0,1,small
9,sep,sat,92.5,88.0,698.6,7.1,22.8,40,4.0,0.0,...,0,0,0,0,0,0,0,0,1,small


In [143]:
#check value of count of columns
forest['size_category'].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [144]:
# cehck all columns name in given Dataset
print(forest.columns)

Index(['month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind',
       'rain', 'area', 'dayfri', 'daymon', 'daysat', 'daysun', 'daythu',
       'daytue', 'daywed', 'monthapr', 'monthaug', 'monthdec', 'monthfeb',
       'monthjan', 'monthjul', 'monthjun', 'monthmar', 'monthmay', 'monthnov',
       'monthoct', 'monthsep', 'size_category'],
      dtype='object')


In [145]:
#check null value
forest.isna().sum()

month            0
day              0
FFMC             0
DMC              0
DC               0
ISI              0
temp             0
RH               0
wind             0
rain             0
area             0
dayfri           0
daymon           0
daysat           0
daysun           0
daythu           0
daytue           0
daywed           0
monthapr         0
monthaug         0
monthdec         0
monthfeb         0
monthjan         0
monthjul         0
monthjun         0
monthmar         0
monthmay         0
monthnov         0
monthoct         0
monthsep         0
size_category    0
dtype: int64

In [146]:
#check value of count of columns
forest['size_category'].value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [147]:
# we need to convert our Target columns using labelencoder
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
forest['size_category']=labelencoder.fit_transform(forest['size_category'])
forest

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,1
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,1
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,1
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,1
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,0
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,0
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,0
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [148]:
#only consider numerical columns
f1=forest.iloc[:,2:10]
f2=forest.iloc[:,30]

In [149]:
# Normalization function 
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [150]:
# Normalized data frame (considering the numerical part of data)
df_norm = norm_func(f1)
df_norm.tail(10)

,FFMC,DMC,DC,ISI,temp,RH,wind,rain
507,0.932903,0.571330,0.873343,0.126560,0.762058,0.305882,0.355556,0.00000
508,0.932903,0.571330,0.873343,0.126560,0.762058,0.305882,0.355556,0.00000
509,0.932903,0.571330,0.873343,0.126560,0.607717,0.658824,0.800000,0.21875
510,0.932903,0.571330,0.873343,0.126560,0.514469,0.552941,0.555556,0.00000
511,0.811613,0.191592,0.771315,0.033868,0.823151,0.235294,0.255556,0.00000
512,0.811613,0.191592,0.771315,0.033868,0.823151,0.200000,0.255556,0.00000
513,0.811613,0.191592,0.771315,0.033868,0.633441,0.658824,0.600000,0.00000
514,0.811613,0.191592,0.771315,0.033868,0.610932,0.647059,0.700000,0.00000
515,0.976774,0.499311,0.711622,0.201426,0.752412,0.317647,0.400000,0.00000
516,0.784516,0.006547,0.115867,0.019608,0.308682,0.188235,0.455556,0.00000


In [151]:
#concat both columns
forest3=pd.concat([df_norm,f2],axis=1)
forest3

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,size_category
0,0.870968,0.086492,0.101325,0.090909,0.192926,0.423529,0.700000,0.00000,1
1,0.927742,0.118194,0.775419,0.119430,0.508039,0.211765,0.055556,0.00000,1
2,0.927742,0.146795,0.796294,0.119430,0.398714,0.211765,0.100000,0.00000,1
3,0.941935,0.110958,0.081623,0.160428,0.196141,0.964706,0.400000,0.03125,1
4,0.910968,0.172984,0.110590,0.171123,0.295820,0.988235,0.155556,0.00000,1
...,...,...,...,...,...,...,...,...,...
512,0.811613,0.191592,0.771315,0.033868,0.823151,0.200000,0.255556,0.00000,0
513,0.811613,0.191592,0.771315,0.033868,0.633441,0.658824,0.600000,0.00000,0
514,0.811613,0.191592,0.771315,0.033868,0.610932,0.647059,0.700000,0.00000,0
515,0.976774,0.499311,0.711622,0.201426,0.752412,0.317647,0.400000,0.00000,1


In [152]:
#split and difine columns
x=forest3.iloc[:,0:8]
y=forest3.iloc[:,8]

In [153]:
x

,FFMC,DMC,DC,ISI,temp,RH,wind,rain
0,0.870968,0.086492,0.101325,0.090909,0.192926,0.423529,0.700000,0.00000
1,0.927742,0.118194,0.775419,0.119430,0.508039,0.211765,0.055556,0.00000
2,0.927742,0.146795,0.796294,0.119430,0.398714,0.211765,0.100000,0.00000
3,0.941935,0.110958,0.081623,0.160428,0.196141,0.964706,0.400000,0.03125
4,0.910968,0.172984,0.110590,0.171123,0.295820,0.988235,0.155556,0.00000
...,...,...,...,...,...,...,...,...
512,0.811613,0.191592,0.771315,0.033868,0.823151,0.200000,0.255556,0.00000
513,0.811613,0.191592,0.771315,0.033868,0.633441,0.658824,0.600000,0.00000
514,0.811613,0.191592,0.771315,0.033868,0.610932,0.647059,0.700000,0.00000
515,0.976774,0.499311,0.711622,0.201426,0.752412,0.317647,0.400000,0.00000


In [154]:
y

0      1
1      1
2      1
3      1
4      1
      ..
512    0
513    0
514    0
515    1
516    1
Name: size_category, Length: 517, dtype: int32

In [155]:
# perform train,train and test size is 0.35. 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.35,stratify=y)

In [156]:
#perform Gridsearch for decide hyper parameter
clf =SVC()
param_grid = [{'kernel':['rbf'],'gamma':[50,5,10,0.5],'C':[15,14,13,12,11,0.1,00.1]}]
gsv =GridSearchCV(clf,param_grid,cv=10)
gsv.fit(x_train,y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [15, 14, 13, 12, 11, 0.1, 0.1],
                          'gamma': [50, 5, 10, 0.5], 'kernel': ['rbf']}])

In [157]:
#let's find best score of C and gamma
gsv.best_params_,gsv.best_score_

({'C': 15, 'gamma': 0.5, 'kernel': 'rbf'}, 0.7469696969696971)

In [158]:
#insiate above values and make a model
clf = SVC(C= 15,gamma=0.5)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
acc = accuracy_score(y_test,y_pred)*100
print("Accuracy =",acc)
confusion_matrix(y_test,y_pred)

Accuracy = 72.92817679558011


array([[  0,  49],
       [  0, 132]], dtype=int64)

In [159]:
#Perform linear as model
model_linear = SVC(kernel = "linear")
model_linear.fit(x_train,y_train)
preds =model_linear.predict(x_test)
print("Accurancy =" ,np.mean(preds==y_test)*100)

Accurancy = 72.92817679558011


In [160]:
#Perform polynomial as model
model_poly = SVC(kernel = "poly")
model_poly.fit(x_train,y_train)
preds =model_poly.predict(x_test)
print("Accurancy =" ,np.mean(preds==y_test)*100)

Accurancy = 72.92817679558011


In [161]:
#Perform sigmoid as model
model_s = SVC(kernel = "sigmoid")
model_s.fit(x_train,y_train)
preds =model_s.predict(x_test)
print("Accurancy =" ,np.mean(preds==y_test)*100)

Accurancy = 71.8232044198895


Q-2
Prepare a classification model using SVM for salary data 

Data Description:

age -- age of a person
workclass	-- A work class is a grouping of work 
education	-- Education of an individuals	
maritalstatus -- Marital status of an individulas	
occupation	 -- occupation of an individuals
relationship -- 	
race --  Race of an Individual
sex --  Gender of an Individual
capitalgain --  profit received from the sale of an investment	
capitalloss	-- A decrease in the value of a capital asset
hoursperweek -- number of hours work per week	
native -- Native of an individual
Salary -- salary of an individual


In [162]:
#import libraby
import pandas as pd
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split ,cross_val_score
import numpy as np

In [163]:
# load dataset
sal_train=pd.read_csv("E:\\DATA SCIENCE\\LMS\\ASSIGNMENT\\MY ASSIGNMENT\\SVM\\SalaryData_Train.csv")
# load dataset
sal_test=pd.read_csv("E:\\DATA SCIENCE\\LMS\\ASSIGNMENT\\MY ASSIGNMENT\\SVM\\SalaryData_Test.csv")

In [164]:
#check value of count of columns
sal_train['Salary'].value_counts()

 <=50K    22653
 >50K      7508
Name: Salary, dtype: int64

In [165]:
#check value of count of columns
sal_test['Salary'].value_counts()

 <=50K    11360
 >50K      3700
Name: Salary, dtype: int64

In [166]:
#check null value
sal_test.isna().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [167]:
#check null value
sal_train.isna().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [175]:
string_col=['workclass','education','maritalstatus','occupation','relationship','race','sex','native']

In [176]:
# we need to convert string columns using labelencoder
from sklearn import preprocessing
label_encoder=preprocessing.LabelEncoder()
for i in string_col:
    sal_train[i]=label_encoder.fit_transform(sal_train[i])
    sal_test[i]=label_encoder.fit_transform(sal_test[i])

In [177]:
sal_train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,5,9,13,4,0,1,4,1,2174,0,40,37,<=50K
1,50,4,9,13,2,3,0,4,1,0,0,13,37,<=50K
2,38,2,11,9,0,5,1,4,1,0,0,40,37,<=50K
3,53,2,1,7,2,5,0,2,1,0,0,40,37,<=50K
4,28,2,9,13,2,9,5,2,0,0,0,40,4,<=50K


In [178]:
#Threre are large dataset so split them.
train_x=sal_train.iloc[0:500,0:13]
train_y=sal_train.iloc[0:500,13]
test_x=sal_test.iloc[0:300,0:13]
test_y=sal_test.iloc[0:300,13]

In [179]:
#SVM Classification using kernels: linear,poly,rbf
from sklearn.svm import SVC

In [180]:
#Perform linear as model
model_linear = SVC(kernel = "linear")
model_linear.fit(train_x,train_y)
preds =model_linear.predict(test_x)
print("Accurancy =" ,np.mean(preds==test_y)*100)

Accurancy = 81.66666666666667


In [181]:
#Perform polynomial as model
model_poly = SVC(kernel = "poly")
model_poly.fit(train_x,train_y)
preds =model_poly.predict(test_x)
print("Accurancy =" ,np.mean(preds==test_y)*100)

Accurancy = 80.33333333333333


In [182]:
#Perform sigmoid as model
model_s = SVC(kernel = "sigmoid")
model_s.fit(train_x,train_y)
preds =model_s.predict(test_x)
print("Accurancy =" ,np.mean(preds==test_y)*100)

Accurancy = 80.0


In [187]:
#kernel=rbf
model_rbf=SVC(kernel='rbf')
model_rbf.fit(train_x,train_y)
test_pred_rbf=model_rbf.predict(test_x)
train_rbf_acc=np.mean(train_pred_rbf==train_y)
print("Accurancy =" ,np.mean(test_pred_rbf==test_y)*100)

Accurancy = 80.33333333333333
